In [1]:
#https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e

import nltk
import numpy as np
import random
import string # to process standard python strings

import warnings
warnings.filterwarnings("ignore")

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# !ls -l /content/gdrive

In [4]:
import pandas as pd

# read the second sheet of the Excel file into a pandas dataframe
# https://www.northwestern.edu/hr/documents/benefits/2023-benefits-guide.pdf
#df = pd.read_excel('/content/gdrive/MyDrive/nu_benefits.xlsx', sheet_name='Sheet1')        # Google Drive
#df = pd.read_excel('c:/users/username/Downloads/nu_benefits.xlsx', sheet_name='Sheet1')    # Windows
df = pd.read_excel('/Users/user/Downloads/nu_benefits.xlsx', sheet_name='Sheet1')           # Mac

#f=open('/Users/user/Downloads/chatbot.txt','r',errors = 'ignore')
#raw=f.read()
#raw=raw.lower()# converts to lowercase

nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
nltk.download('stopwords') # first-time use only

#page_tokens = nltk.sent_tokenize(raw)# converts to list of pages
from_xlsx = df['Text'].tolist()
for i in range(len(from_xlsx)):
    from_xlsx[i] = from_xlsx[i].replace("\n", " ")

all_text = " ".join(from_xlsx)
raw= all_text.lower()
#s
# page_tokens are tokens for the entire paragraph or page.
page_tokens = from_xlsx
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /Users/barrychiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/barrychiu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/barrychiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# https://stackoverflow.com/questions/17390326/getting-rid-of-stop-words-and-document-tokenization-using-nltk
from nltk.corpus import stopwords
from nltk import word_tokenize


lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    stop = set(stopwords.words('english') + list(string.punctuation))
    text2= ' '.join([i for i in word_tokenize(text.lower()) if i not in stop])
    return LemTokens(nltk.word_tokenize(text2.lower().translate(remove_punct_dict)))

In [6]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# https://stackoverflow.com/questions/17390326/getting-rid-of-stop-words-and-document-tokenization-using-nltk


def response(user_response):
    robo_response=''
    # TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVec.fit_transform(page_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        # found page, now search for a sentence on the page
        TfidfVec2 = TfidfVectorizer(tokenizer=LemNormalize)
        #print(page_tokens[idx])
        specific_sent_tokens2 = nltk.sent_tokenize(page_tokens[idx])
        specific_sent_tokens2.append(user_response)
        #word_tokens=word_tokens+nltk.word_tokenize(user_response)
        tfidf2 = TfidfVec2.fit_transform(specific_sent_tokens2)     
        vals2 = cosine_similarity(tfidf2[-1], tfidf2)
        idx2=vals2.argsort()[0][-2]
        flat2 = vals2.flatten()
        flat2.sort()
        req_tfidf2 = flat2[-2]
        if(req_tfidf2==0):
            robo_response=robo_response+"I am sorry! I don't understand you"
            return robo_response        
        robo_response = robo_response+specific_sent_tokens2[idx2]
        return robo_response

In [9]:
flag=True
print("\nROBO: My name is Robo. I will answer your queries. If you want to exit, type Bye")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("\nROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("\nROBO: "+greeting(user_response))
            else:
                page_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print()
                print("\nROBO: ",end="")
                print(response(user_response))
                page_tokens.remove(user_response)
                print()
    else:
        flag=False
        print("\nROBO: Bye! take care..")


ROBO: My name is Robo. I will answer your queries. If you want to exit, type Bye


 What PPOs are available?




ROBO: Northwestern offers four medical plans: §	Premier PPO, Select PPO, and Value PPO – three preferred provider options (PPOs) that offer the flexibility to go to any provider you choose (though you will receive greater benefits and pay lower out-of-pocket costs when you see a provider who is a member of the Northwestern Medicine or BlueCross BlueShield PPO Network).



 Bye



ROBO: Bye! take care..


In [ ]:
# What is the employee reduced tuition benefit?

# What is reduced tuition benefit for my spouse for undergraduate classes?

# What PPOs are available?

# When am I eligible for benefits?

